In [1]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.sparse

In [2]:
import anndata2ri
from rpy2.robjects import r

In [3]:
anndata2ri.activate()

In [4]:
r(f"library(Seurat)\n")


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject



array(['SeuratObject', 'Seurat', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U12')

In [5]:
def harmonise_genespace(adata, genes, keep_layer=[]):
    genes_to_add = []
    for g in genes:
        if g not in adata.var_names:
            genes_to_add += [g]
    print(f"{(len(genes)-len(genes_to_add))/len(genes)*100:.0f} % gene overlap with reference")
    adata = ad.AnnData(
        X=scipy.sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape=(adata.shape[0], adata.shape[1] + len(genes_to_add))),
        obs=adata.obs,
        var=pd.DataFrame(index=adata.var.index.tolist() + genes_to_add),
        layers={i:scipy.sparse.csr_matrix((adata.layers[i].data, adata.layers[i].indices, adata.layers[i].indptr), shape=(adata.shape[0], adata.shape[1] + len(genes_to_add))) for i in keep_layer}
    )[:, genes].copy()
    
    return adata

Preprint: https://www.biorxiv.org/content/10.1101/2023.07.11.548571

Data provided directly by the authors.

In [6]:
basepath = "/storage/data/2404_revision/ce_data/bowles_data"

In [7]:
for fn in ['2021_Batch1_annotated', 'Batch2', 'Batch3', 'BFsubset']:
    r(
        f"so = readRDS('{basepath}/{fn}.rds')\n"
        f"as.SingleCellExperiment(so, assay='RNA')\n"
    ).write(f"{basepath}/{fn}.h5ad", compression='gzip')

/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata2ri/r2py.py:106: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata2ri/r2py.py:106: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata2ri/r2py.py:106: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not

In [8]:
adatas = []
for fn in ['2021_Batch1_annotated', 'Batch2', 'Batch3', 'BFsubset']:
    adatas.append(ad.read_h5ad(f"{basepath}/{fn}.h5ad"))

In [9]:
adatas[0].obs['sample'] = 'batch0_' + adatas[0].obs['sample'].astype(str)
adatas[1].obs['sample'] = 'batch1_' + adatas[1].obs['sample'].astype(str)
adatas[2].obs['sample'] = 'batch2_'
adatas[3].obs['sample'] = 'batch3_'

In [10]:
adatas

[AnnData object with n_obs × n_vars = 26327 × 36601
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'percent.mito', 'RNA_snn_res.0.6', 'seurat_clusters', 'RNA_snn_res.0.3', 'ParentLine', 'CellLine', 'Mt', 'Age', 'Batch', 'Run', 'HTO', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.5', 'integrated_snn_res.0.5', 'CellType', 'QC', 'ident'
     obsm: 'X_pca', 'X_umap'
     layers: 'logcounts',
 AnnData object with n_obs × n_vars = 33753 × 36601
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'percent.mito', 'RNA_snn_res.0.6', 'seurat_clusters', 'RNA_snn_res.0.3', 'ParentLine', 'CellLine', 'Mt', 'Age', 'Batch', 'Run', 'HTO', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.5', 'CellType', 'MixedAnnotation', 'QC', 'ident'
     obsm: 'X_pca', 'X_umap'
     layers: 'logcounts',
 AnnData object with n_obs × n_vars = 26147 × 36601
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'seurat_clusters', 'RNA_snn_res.0.6'

In [11]:
adata = ad.concat(adatas, index_unique="_", label="Batch")

In [12]:
adata = adata[adata.obs["Mt"] == "Control"].copy()

In [13]:
hnoca_genes = pd.read_csv("/storage/data/2404_revision/hnoca_features.csv", index_col=0)
adata = harmonise_genespace(adata, hnoca_genes.index.tolist())
adata.var = hnoca_genes

63 % gene overlap with reference


In [14]:
obs_new = pd.DataFrame(index=adata.obs.index)

obs_new["sample_source"] = "3d_culture"
obs_new["organism"] = "Homo sapiens"
obs_new["disease"] = "healthy"

obs_new["cell_line_original"] = adata.obs["CellLine"]
obs_new["organoid_age_days"] = 60
obs_new["batch"] = adata.obs["sample"]
obs_new["treatment"] = adata.obs["Mt"]
obs_new["cell_type_original"] = adata.obs["CellType"]

obs_new["suspension_type"] = "cell"
obs_new["organ"] = "cerebral cortex"
obs_new["assay_sc"] = "10x 3' v3"
obs_new["ethnicity"] = "unknown"
obs_new["sex"] = "unknown"
obs_new["development_stage"] = "unknown"
obs_new["cell_type"] = obs_new["cell_type_original"].replace({
    'ExDp2': 'glutamatergic neuron',
    'Glia': 'glial cell',
    'UN': 'neuron',
    'PgG2M': 'neural progenitor cell',
    'vRG': 'radial glial cell',
    'Ast': 'astrocyte',
    'ExDp1': 'glutamatergic neuron',
    'ExM-U': 'glutamatergic neuron',
    'oRG': 'radial glial cell',
    'IP': 'neural progenitor cell',
    'PgS': 'neural progenitor cell',
    'InMGE': 'medial ganglionic eminence derived GABAergic cortical interneuron',
    'ExM': 'glutamatergic neuron',
    'ExN': 'glutamatergic neuron',
    'InCGE': 'caudal ganglionic eminence derived GABAergic cortical interneuron',
    'OPC': 'oligodendrocyte precursor cell',
})

obs_new["obs_names_original"] = obs_new.index
obs_new["publication"] = "Bertucci, 2023"
obs_new["doi"] = "10.1101/2023.07.11.548571"
obs_new["hnoca_core"] = False

In [15]:
adata.obs = obs_new

adata.layers["counts_lengthnorm"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

In [16]:
ncells_before = adata.n_obs
adata = adata[(adata.X>0).sum(axis=1).A.ravel()>200].copy()
print(f"{(ncells_before-adata.n_obs)/ncells_before*100:.1f} % cells removed")

0.0 % cells removed


In [17]:
adata

AnnData object with n_obs × n_vars = 45763 × 36842
    obs: 'sample_source', 'organism', 'disease', 'cell_line_original', 'organoid_age_days', 'batch', 'treatment', 'cell_type_original', 'suspension_type', 'organ', 'assay_sc', 'ethnicity', 'sex', 'development_stage', 'cell_type', 'obs_names_original', 'publication', 'doi', 'hnoca_core'
    var: 'ensembl', 'gene_symbol'
    uns: 'log1p'
    layers: 'counts_lengthnorm'

In [18]:
adata.write(f"{basepath}/bowles.h5ad", compression='gzip')